In [1]:
import sys
import random
import math
import operator
import os

In [2]:
import numpy as np  # using numpy package for mathematical operations
from numpy import *
from numpy.linalg import inv  # used for inversing the matrix
import numpy.ma

In [3]:
from random import randint
import math
import csv
from pyspark import SparkContext

from operator import add

import re
import string
import pandas as pd

In [4]:
from collections import Counter

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.probability import FreqDist as nF
from nltk.corpus import stopwords
from nltk import bigrams
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [5]:
stop_words = []
data_set = []
decision_attributes = []
tokens_re = ""

In [6]:
# Other required global variables
rating_list = []
vocab_len = 0
n_one = 0
n_two = 0
n_three = 0
n_four = 0
n_five = 0
n_six = 0
n_seven = 0
n_eight = 0
n_nine = 0
n_ten = 0
n = 0

words_in_one = {}
words_in_two = {}
words_in_three = {}
words_in_four = {}
words_in_five = {}
words_in_six = {}
words_in_seven = {}
words_in_eight = {}
words_in_nine = {}
words_in_ten = {}


class_probability = {}

In [7]:
sc = SparkContext(appName="Naive-Bayes")

In [8]:
def load_file(file_path):
    print("Loading the dataset...")
    file_read = open(file_path, "r")
    reader = csv.reader(file_read, dialect='excel')
    next(reader, None)
    for row in reader:
        temp = (row[6], row[9], row[12])
        
        data_set.append(temp)

In [9]:
load_file("Hotel_Reviews.csv")

Loading the dataset...


In [10]:
train_data_features_reviews = pd.DataFrame(data_set, columns=["Negative_Review", "Positive_Review", "Reviewer_Score"])
target_review = train_data_features_reviews['Reviewer_Score']

train_data_features_reviews['Review'] = train_data_features_reviews['Negative_Review'] + " " + train_data_features_reviews['Positive_Review']


In [11]:
train_data_features_reviews.Reviewer_Score = pd.to_numeric(train_data_features_reviews.Reviewer_Score, errors='ignore')
train_data_features_reviews.Reviewer_Score = pd.to_numeric(train_data_features_reviews.Reviewer_Score, errors='coerce')

train_data_features_reviews.Reviewer_Score = train_data_features_reviews.Reviewer_Score.astype(int)

In [12]:
# Emoticons string if any for removal
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""

# Regex for non alphanumeric characters
regex_str = [
    r'<[^>]+>',  # HTML tags
    r"(?:[a-z][a-z\-_]+[a-z])",  # words with - and '
    r'(?:[\w_]+)',  # other words
    r'(?:\S)'  # anything else
]

In [13]:
# Prepare the dataset to remove to remove stop words by intializing the system
def initializeSystem():
    print("Preparing stop words.")
    stop = set(stopwords.words('english') + punctuation + ['rt', 'via', 'i\'m', 'us', 'it'])
    for x in stop:
        stop_words.append(stemmer.stem(lemmatiser.lemmatize(x, pos="v")))

In [14]:
# initializing NLTK functions for preprocessing
tokens_re = re.compile(r'(' + '|'.join(regex_str) + ')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^' + emoticons_str + '$', re.VERBOSE | re.IGNORECASE)
punctuation = list(string.punctuation)
stemmer = PorterStemmer()
lemmatiser = WordNetLemmatizer()
initializeSystem()

Preparing stop words.


In [15]:
print("Preprocess the Data")

# Tokenize will separate each word as tokens
def tokenize(s):
    return tokens_re.findall(s)

# This function removes non-alhanumeric characters, removes the emoticons if present and stems and lemmatize each word
def preprocessing(s, lowercase=True):
    s = re.sub('[^a-zA-Z0-9\n\.]', ' ', s)
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else stemmer.stem(lemmatiser.lemmatize(token.lower(), pos="v")) for
                  token in tokens]
    return tokens


# Each row is broken down into space separated words or terms of list and sent for stemming and lemming. Here stop words are removed
def process_string(string):
    term_stop = [term for term in preprocessing(string) if term not in stop_words and len(str(term)) > 1]
    return term_stop

# This function is used to send each rows to preprocessing and then combines the result back to a single dataset
def preprocess_reviews(reviews):
    preprocessed_reviews = []
    for index, row in reviews.iterrows():
        temp = process_string(str(row["Review"]).lower())
        str1 = ' '.join(temp)
        temp = [str1]
        temp.append(row["Reviewer_Score"])
        preprocessed_reviews.append(temp)
    return preprocessed_reviews

train_reviews = preprocess_reviews(train_data_features_reviews)

Preprocess the Data


In [16]:
yx_lines = sc.parallelize(train_reviews)
train, test = yx_lines.randomSplit([0.6, 0.4])


In [17]:
print("Building the Naive Bayes Model")
words_in_train_data1 = train.map(lambda x: tuple(x)).map(lambda x: x[0].split(' '))
words_in_train_data2 = [y for x in words_in_train_data1.collect() for y in x]

Building the Naive Bayes Model


In [18]:
words_in_train_data = Counter(words_in_train_data2)
vocab_len = len(words_in_train_data)
n = train.count()

In [19]:
train.first()

['neg real complaint hotel great great locat surround room amen servic two recommend howev firstli staff upon check confus regard deposit payment staff offer upon checkout refund origin payment make new one bite confus secondli site restaur bite lack well think excel qualiti food anyon vegetarian vegan background even wrap toast sandwich option would great asid minor minor thing fantast spot back return amsterdam',
 7]

In [20]:
rating_review_list = train.map(lambda x: (x[1], x[0])).groupByKey().map(lambda x: (x[0],tuple(x[1]))).persist()

In [21]:
#print(rating_review_list.first())

In [22]:
# Calculating number of words in each class

words_in_one1 = sc.parallelize(rating_review_list.lookup(1)).flatMap(lambda xs: [(x, 1) for x in xs]).map(lambda x: x[0].split(' '))
words_in_one2 = [y for x in words_in_one1.collect() for y in x]
words_in_one = Counter(words_in_one2)
n_one = len(words_in_one)


In [23]:
words_in_two1 = sc.parallelize(rating_review_list.lookup(2)).flatMap(lambda xs: [(x, 1) for x in xs]).map(lambda x: x[0].split(' '))
words_in_two2 = [y for x in words_in_two1.collect() for y in x]
words_in_two = Counter(words_in_two2)
n_two = len(words_in_two2)

In [24]:
words_in_three1 = sc.parallelize(rating_review_list.lookup(3)).flatMap(lambda xs: [(x, 1) for x in xs]).map(
lambda x: x[0].split(' '))
words_in_three2 = [y for x in words_in_three1.collect() for y in x]
words_in_three = Counter(words_in_three2)
n_three = len(words_in_three)

In [25]:
words_in_four1 = sc.parallelize(rating_review_list.lookup(4)).flatMap(lambda xs: [(x, 1) for x in xs]).map(
lambda x: x[0].split(' '))
words_in_four2 = [y for x in words_in_four1.collect() for y in x]
words_in_four = Counter(words_in_four2)
n_four = len(words_in_four)

In [26]:
words_in_five1 = sc.parallelize(rating_review_list.lookup(5)).flatMap(lambda xs: [(x, 1) for x in xs]).map(
lambda x: x[0].split(' '))
words_in_five2 = [y for x in words_in_five1.collect() for y in x]
words_in_five = Counter(words_in_five2)
n_five = len(words_in_five)

In [27]:
words_in_six1 = sc.parallelize(rating_review_list.lookup(6)).flatMap(lambda xs: [(x, 1) for x in xs]).map(
lambda x: x[0].split(' '))
words_in_six2 = [y for x in words_in_six1.collect() for y in x]
words_in_six = Counter(words_in_six2)
n_six = len(words_in_six)

In [28]:
words_in_seven1 = sc.parallelize(rating_review_list.lookup(7)).flatMap(lambda xs: [(x, 1) for x in xs]).map(
lambda x: x[0].split(' '))
words_in_seven2 = [y for x in words_in_seven1.collect() for y in x]
words_in_seven = Counter(words_in_seven2)
n_seven = len(words_in_seven)

In [29]:
words_in_eight1 = sc.parallelize(rating_review_list.lookup(8)).flatMap(lambda xs: [(x, 1) for x in xs]).map(
lambda x: x[0].split(' '))
words_in_eight2 = [y for x in words_in_eight1.collect() for y in x]
words_in_eight = Counter(words_in_eight2)
n_eight = len(words_in_eight)

In [30]:
words_in_nine1 = sc.parallelize(rating_review_list.lookup(9)).flatMap(lambda xs: [(x, 1) for x in xs]).map(
lambda x: x[0].split(' '))
words_in_nine2 = [y for x in words_in_nine1.collect() for y in x]
words_in_nine = Counter(words_in_nine2)
n_nine = len(words_in_nine)

In [31]:
words_in_ten1 = sc.parallelize(rating_review_list.lookup(10)).flatMap(lambda xs: [(x, 1) for x in xs]).map(
lambda x: x[0].split(' '))
words_in_ten2 = [y for x in words_in_ten1.collect() for y in x]
words_in_ten = Counter(words_in_ten2)
n_ten = len(words_in_ten)

In [32]:
#words_in_nine

In [33]:
def get_prob(words):
    orig_rating = int(words[0])
    words = words[1:]

    probability1 = 0.0 + math.log10(class_probability.get(1, 1.0))
    probability2 = 0.0 + math.log10(class_probability.get(2, 1.0))
    probability3 = 0.0 + math.log10(class_probability.get(3, 1.0))
    probability4 = 0.0 + math.log10(class_probability.get(4, 1.0))
    probability5 = 0.0 + math.log10(class_probability.get(5, 1.0))
    probability6 = 0.0 + math.log10(class_probability.get(6, 1.0))
    probability7 = 0.0 + math.log10(class_probability.get(7, 1.0))
    probability8 = 0.0 + math.log10(class_probability.get(8, 1.0))
    probability9 = 0.0 + math.log10(class_probability.get(9, 1.0))
    probability10 = 0.0 + math.log10(class_probability.get(10, 1.0))

    final_prob = 0

    for i in range(len(words)):
        if (words[i] in words_in_one):
            probability1 = probability1 + math.log10(words_in_one[words[i]])

        if (words[i] in words_in_two):
            probability2 = probability2 + math.log10(words_in_two[words[i]])

        if (words[i] in words_in_three):
            probability3 = probability3 + math.log10(words_in_three[words[i]])

        if (words[i] in words_in_four):
            probability4 = probability4 + math.log10(words_in_four[words[i]])

        if (words[i] in words_in_five):
            probability5 = probability5 + math.log10(words_in_five[words[i]])
            
        if (words[i] in words_in_six):
            probability6 = probability6 + math.log10(words_in_six[words[i]])

        if (words[i] in words_in_seven):
            probability7 = probability7 + math.log10(words_in_seven[words[i]])

        if (words[i] in words_in_eight):
            probability8 = probability8 + math.log10(words_in_eight[words[i]])

        if (words[i] in words_in_nine):
            probability9 = probability9 + math.log10(words_in_nine[words[i]])

        if (words[i] in words_in_ten):
            probability10 = probability10 + math.log10(words_in_ten[words[i]])

    max_prob = probability2
    final_prob = 2

    if (probability1 == 0.0 and probability2 == 0.0 and probability3 == 0.0 and probability4 == 0.0 and probability5 == 0.0 and probability6 == 0.0 and probability7 == 0.0 and probability8 == 0.0 and probability9 == 0.0 and probability10 == 0.0):
        # finalprob=randint(1,5)
        final_prob = 0
    else:
#         if (probability2 > max_prob):
#             max_prob = probability2
#             final_prob = 2
        if (probability3 > max_prob):
            max_prob = probability3
            final_prob = 3
        if (probability4 > max_prob):
            max_prob = probability4
            final_prob = 4
        if (probability5 > max_prob):
            max_prob = probability5
            final_prob = 5
        if (probability6 > max_prob):
            max_prob = probability6
            final_prob = 6
        if (probability7 > max_prob):
            max_prob = probability7
            final_prob = 7
        if (probability8 > max_prob):
            max_prob = probability8
            final_prob = 8
        if (probability9 > max_prob):
            max_prob = probability9
            final_prob = 9
        if (probability10 > max_prob):
            max_prob = probability10
            final_prob = 10

    return (orig_rating, final_prob)

In [34]:
# this function is used to calculate the 
def get_accuracy(predictions):
    correct = 0
    for x in range(len(predictions)):
        if predictions[x][0] == predictions[x][1]:
            correct += 1
    return (correct / float(len(predictions))) * 100.0

def get_userreview_prob(words):
  
    probability2 = 0.0 + math.log10(class_probability.get(2, 1.0))
    probability3 = 0.0 + math.log10(class_probability.get(3, 1.0))
    probability4 = 0.0 + math.log10(class_probability.get(4, 1.0))
    probability5 = 0.0 + math.log10(class_probability.get(5, 1.0))
    
    probability6 = 0.0 + math.log10(class_probability.get(6, 1.0))
    probability7 = 0.0 + math.log10(class_probability.get(7, 1.0))
    probability8 = 0.0 + math.log10(class_probability.get(8, 1.0))
    probability9 = 0.0 + math.log10(class_probability.get(9, 1.0))
    probability10 = 0.0 + math.log10(class_probability.get(10, 1.0))

    final_prob = 0

    for i in range(len(words)):

        if (words[i] in words_in_two):
            probability2 = probability2 + math.log10(words_in_two[words[i]])

        if (words[i] in words_in_three):
            probability3 = probability3 + math.log10(words_in_three[words[i]])

        if (words[i] in words_in_four):
            probability4 = probability4 + math.log10(words_in_four[words[i]])

        if (words[i] in words_in_five):
            probability5 = probability5 + math.log10(words_in_five[words[i]])
            
        if (words[i] in words_in_six):
            probability6 = probability6 + math.log10(words_in_six[words[i]])

        if (words[i] in words_in_seven):
            probability7 = probability7 + math.log10(words_in_seven[words[i]])

        if (words[i] in words_in_eight):
            probability8 = probability8 + math.log10(words_in_eight[words[i]])

        if (words[i] in words_in_nine):
            probability9 = probability9 + math.log10(words_in_nine[words[i]])

        if (words[i] in words_in_ten):
            probability10 = probability10 + math.log10(words_in_ten[words[i]])
             
    max_prob = probability2
    final_prob = 1
    
    print(max_prob,probability2,probability3,probability4,probability5,probability6,probability7,probability8,probability9,probability10)
    
    if (probability2 == 0.0 and probability3 == 0.0 and probability4 == 0.0 
        and probability5 == 0.0 and probability6 == 0.0 and probability7 == 0.0 and probability8 == 0.0 and
       probability9 == 0.0 and probability10 == 0.0):
         finalprob=randint(1,5)
        #final_prob = 0
    else:
        if (probability3 > max_prob):
            max_prob = probability3
            final_prob = 3
        if (probability4 > max_prob):
            max_prob = probability4
            final_prob = 4
        if (probability5 > max_prob):
            max_prob = probability5
            final_prob = 5
        if (probability6 > max_prob):
            max_prob = probability6
            final_prob = 6
        if (probability7 > max_prob):
            max_prob = probability7
            final_prob = 7
        if (probability8 > max_prob):
            max_prob = probability8
            final_prob = 8
        if (probability9 > max_prob):
            max_prob = probability9
            final_prob = 9
        if (probability10 > max_prob):
            max_prob = probability10
            final_prob = 10

    return final_prob

def pred_rat(s):
    test_prediction = sc.parallelize([s])
    testwords = test_prediction.map(lambda x: tuple(x)).map(lambda x: get_userreview_prob(x))
    pred = testwords.first()
    print("The predicted rating from NaiveBayes for this review is",pred)

In [35]:
alpha=0.5
alpha_vocab = alpha * vocab_len

words_in_one = dict(sc.parallelize(words_in_one.items()).map(lambda x: (x[0], np.divide(np.add(x[1], alpha), np.add(n_one, alpha_vocab)))).collect())

words_in_two = dict(sc.parallelize(words_in_two.items()).map(lambda x: (x[0], np.divide(np.add(x[1], alpha), np.add(n_two, alpha_vocab)))).collect())

words_in_three = dict(sc.parallelize(words_in_three.items()).map(lambda x: (x[0], np.divide(np.add(x[1], alpha), np.add(n_three, alpha_vocab)))).collect())

words_in_four = dict(sc.parallelize(words_in_four.items()).map(lambda x: (x[0], np.divide(np.add(x[1], alpha), np.add(n_four, alpha_vocab)))).collect())

words_in_five = dict(sc.parallelize(words_in_five.items()).map(lambda x: (x[0], np.divide(np.add(x[1], alpha), np.add(n_five, alpha_vocab)))).collect())

words_in_six = dict(sc.parallelize(words_in_six.items()).map(lambda x: (x[0], np.divide(np.add(x[1], alpha), np.add(n_six, alpha_vocab)))).collect())

words_in_seven = dict(sc.parallelize(words_in_seven.items()).map(lambda x: (x[0], np.divide(np.add(x[1], alpha), np.add(n_seven, alpha_vocab)))).collect())

words_in_eight = dict(sc.parallelize(words_in_eight.items()).map(lambda x: (x[0], np.divide(np.add(x[1], alpha), np.add(n_eight, alpha_vocab)))).collect())

words_in_nine = dict(sc.parallelize(words_in_nine.items()).map(lambda x: (x[0], np.divide(np.add(x[1], alpha), np.add(n_nine, alpha_vocab)))).collect())

words_in_ten = dict(sc.parallelize(words_in_ten.items()).map(lambda x: (x[0], np.divide(np.add(x[1], alpha), np.add(n_ten, alpha_vocab)))).collect())

class_probability = dict(rating_review_list.map(lambda x: (x[0], len([item for item in x[-1] if item]))).map(lambda x: (x[0], (x[1] * 1.0) / n)).collect())

print("Testing the Naive Bayes Model")

test_words1 = test.map(lambda x: (x[1], str(x[1]) + " " + x[0])).map(lambda x: (x[0], (x[1:]))).map(lambda x:x[1]) \
.flatMap(lambda xs: [(x, 1) for x in xs]) \
.map(lambda x: x[0].split(' ')) \
.map(lambda x: get_prob(x))

predictions = test_words1.collect()


accuracy = get_accuracy(predictions)

print("#################")
print('Accuracy: with alpha value for NaiveBayes '+ repr(alpha)+ ' is ' + repr(accuracy) + '%')

Testing the Naive Bayes Model
#################
Accuracy: with alpha value for NaiveBayes 0.5 is 30.00363645179277%


In [37]:
pred_rat("beautiful view great restaurant awesome food")

The predicted rating from NaiveBayes for this review is 9
